# 분류 헤드

모든 🤗 SetFit 모델은 [SentenceTransformer](https://sbert.net/) 임베딩 본문과 분류 헤드의 두 부분으로 구성됩니다. 

이 가이드에서는 다음을 보여줍니다.
* 내장 로지스틱 회귀 분류 헤드
* 내장 미분 가능 분류 헤드
* 사용자 지정 분류 헤드의 요구 사항

## 로지스틱 회귀 분류 헤드

새로운 SetFit 모델이 초기화되면 기본적으로 [scikit-learn 로지스틱 회귀](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) 헤드가 선택됩니다. 이는 미세 조정된 문장 변환기 본문 위에 적용될 때 매우 효과적인 것으로 나타났으며 권장되는 분류 헤드로 남아 있습니다. 로지스틱 회귀 헤드로 새 SetFit 모델을 초기화하는 것은 간단합니다.

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5")
model.model_head

LogisticRegression()

로지스틱 회귀 헤드(또는 다른 헤드)를 추가 매개변수로 초기화하려면 `SetFitModel.from_pretrained()`에서 `head_params` 인수를 사용할 수 있습니다.

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", head_params={"solver": "liblinear", "max_iter": 300})
model.model_head

LogisticRegression(max_iter=300, solver='liblinear')

## 미분 가능 분류 헤드

SetFit은 또한 독점적으로 `torch` 분류 헤드인 [SetFitHead](https://huggingface.co/docs/setfit/main/en/reference/main#setfit.SetFitHead)를 제공합니다. 선형 계층을 사용하여 임베딩을 클래스에 매핑합니다. `SetFitModel.from_pretrained()`에서 `use_differentiable_head` 인수를 `True`로 설정하여 사용할 수 있습니다.

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", use_differentiable_head=True)
model.model_head

SetFitHead({'in_features': 384, 'out_features': 2, 'temperature': 1.0, 'bias': True, 'device': 'cuda'})

기본적으로 이진 분류를 가정합니다. 이를 변경하려면 `head_params`를 통해 `out_features`를 사용 중인 클래스 수로 설정합니다.

In [ ]:
from setfit import SetFitModel

model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", use_differentiable_head=True, head_params={"out_features": 5})
model.model_head

SetFitHead({'in_features': 384, 'out_features': 5, 'temperature': 1.0, 'bias': True, 'device': 'cuda'})

<Tip warning={true}>

기본 로지스틱 회귀 헤드와 달리 미분 가능 분류 헤드는 다음 범위의 정수 레이블만 지원합니다. `[0, num_classes)`.

</Tip>

### 미분 가능 분류 헤드로 훈련

[SetFitHead](https://huggingface.co/docs/setfit/main/en/reference/main#setfit.SetFitHead)를 사용하면 sklearn 기반 헤드와 함께 사용되지 않는 새로운 [TrainingArguments](https://huggingface.co/docs/setfit/main/en/reference/trainer#setfit.TrainingArguments)가 잠금 해제됩니다. SetFit으로 훈련하는 것은 내부적으로 **임베딩 미세 조정**과 **분류 헤드 훈련**의 두 단계로 구성됩니다. 결과적으로 일부 훈련 인수는 튜플일 수 있으며, 두 값은 각각 두 단계에 사용됩니다. 이러한 경우 대부분 분류 헤드가 미분 가능해야 두 번째 값이 사용됩니다. 예를 들어:

* **batch_size**: (`Union[int, Tuple[int, int]]`, 기본값: `(16, 2)`) - 튜플의 두 번째 값은 미분 가능 SetFitHead를 훈련할 때 배치 크기를 결정합니다.
* **num_epochs**: (`Union[int, Tuple[int, int]]`, 기본값: `(1, 16)`) - 튜플의 두 번째 값은 미분 가능 SetFitHead를 훈련할 때 에포크 수를 결정합니다. 실제로 `num_epochs`는 일반적으로 분류 헤드 훈련에 더 큽니다. 여기에는 두 가지 이유가 있습니다.

    1. 이 훈련 단계에서는 대조 쌍으로 훈련하지 않으므로 임베딩 모델을 미세 조정할 때와 달리 레이블이 지정된 각 훈련 텍스트에 대해 하나의 훈련 샘플만 얻습니다.
    2. 이 훈련 단계에는 이미 유능한 모델을 미세 조정하는 것이 아니라 처음부터 분류기를 훈련하는 것이 포함됩니다. 이를 위해 더 많은 훈련 단계가 필요합니다.
* **end_to_end**: (`bool`, 기본값: `False`) - `True`이면 분류기 훈련 단계에서 전체 모델을 종단 간으로 훈련합니다. 그렇지 않으면 Sentence Transformer 본문을 고정하고 헤드만 훈련합니다.
* **body_learning_rate**: (`Union[float, Tuple[float, float]]`, 기본값: `(2e-5, 1e-5)`) - 튜플의 두 번째 값은 분류기 훈련 단계에서 Sentence Transformer 본문의 학습률을 결정합니다. `end_to_end`가 `True`인 경우에만 관련이 있습니다. 그렇지 않으면 분류기를 훈련할 때 Sentence Transformer 본문이 고정됩니다.
* **head_learning_rate** (`float`, 기본값: `1e-2`) - 이 값은 분류기 훈련 단계에서 미분 가능 헤드의 학습률을 결정합니다. 미분 가능 헤드가 사용되는 경우에만 사용됩니다.
* **l2_weight** (`float`, *선택 사항*) - 모델 본문과 헤드 모두에 대한 선택적 l2 가중치로, 미분 가능 헤드가 사용되는 경우에만 분류기 훈련 단계에서 `AdamW` 옵티마이저에 전달됩니다.

예를 들어 미분 가능 분류 헤드를 사용하는 전체 훈련 스크립트는 다음과 같을 수 있습니다.

In [ ]:
from setfit import SetFitModel, Trainer, TrainingArguments, sample_dataset
from datasets import load_dataset

# 새로운 SetFit 모델 초기화
model = SetFitModel.from_pretrained("BAAI/bge-small-en-v1.5", use_differentiable_head=True, head_params={"out_features": 2})

# 데이터셋 준비
dataset = load_dataset("SetFit/sst2")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=32)
test_dataset = dataset["test"]

# 훈련 인수 준비
args = TrainingArguments(
    batch_size=(32, 16),
    num_epochs=(3, 8),
    end_to_end=True,
    body_learning_rate=(2e-5, 5e-6),
    head_learning_rate=2e-3,
    l2_weight=0.01,
)

# 트레이너 준비
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)
trainer.train()
# ***** 훈련 실행 중 *****
#   예제 수 = 66
#   에포크 수 = 3
#   총 최적화 단계 = 198
#   총 훈련 배치 크기 = 3
# {'embedding_loss': 0.2204, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.02}                                                                                 
# {'embedding_loss': 0.0058, 'learning_rate': 1.662921348314607e-05, 'epoch': 0.76}                                                                                  
# {'embedding_loss': 0.0026, 'learning_rate': 1.101123595505618e-05, 'epoch': 1.52}                                                                                  
# {'embedding_loss': 0.0022, 'learning_rate': 5.393258426966292e-06, 'epoch': 2.27}                                                                                  
# {'train_runtime': 36.6756, 'train_samples_per_second': 172.758, 'train_steps_per_second': 5.399, 'epoch': 3.0}                                                     
# 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 198/198 [00:30<00:00,  6.45it/s] 
# `max_length`는 `None`입니다. 현재 모델 본문에 따라 허용되는 최대 길이 사용: 512.
# 에포크: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 8/8 [00:07<00:00,  1.03it/s]

# 평가
metrics = trainer.evaluate(test_dataset)
print(metrics)
# => {'accuracy': 0.8632619439868204}

# 추론 수행
preds = model.predict([
    "매력적이고 종종 감동적인 여정입니다.",
    "매우, 매우 느립니다.",
    "때로는 지루한 영화입니다.",
])
print(preds)
# => tensor([1, 0, 0], device='cuda:0')

## 사용자 지정 분류 헤드

두 가지 내장 옵션 외에도 SetFit을 사용하면 사용자 지정 분류 헤드를 지정할 수 있습니다. 지원되는 헤드에는 사용자 지정 **미분 가능** 헤드 또는 사용자 지정 **미분 불가능** 헤드의 두 가지 형태가 있습니다. 두 헤드 모두 다음 두 가지 메서드를 구현해야 합니다.

### 사용자 지정 미분 가능 헤드

사용자 지정 미분 가능 헤드는 다음 요구 사항을 따라야 합니다.

* `nn.Module`을 서브클래싱해야 합니다.
* `predict` 메서드: `(self, torch.Tensor with shape [num_inputs, embedding_size]) -> torch.Tensor with shape [num_inputs]` - 이 메서드는 임베딩을 분류합니다. 출력은 `[0, num_classes)` 범위의 정수여야 합니다.
* `predict_proba` 메서드: `(self, torch.Tensor with shape [num_inputs, embedding_size]) -> torch.Tensor with shape [num_inputs, num_classes]` - 이 메서드는 임베딩을 각 클래스에 대한 확률로 분류합니다. 각 입력에 대해 크기가 `num_classes`인 텐서는 합계가 1이어야 합니다. `torch.argmax(output, dim=-1)`을 적용하면 `predict`에 대한 출력이 생성됩니다.
* `get_loss_fn` 메서드: `(self) -> nn.Module` - 초기화된 손실 함수(예: `torch.nn.CrossEntropyLoss()`)를 반환합니다.
* `forward` 메서드: `(self, Dict[str, torch.Tensor]) -> Dict[str, torch.Tensor]` - Sentence Transformer 본문의 출력, 즉 `'input_ids'`, `'token_type_ids'`, `'attention_mask'`, `'token_embeddings'` 및 `'sentence_embedding'` 키의 사전을 지정하면 `'logits'` 키와 모양이 `[batch_size, num_classes]`인 `torch.Tensor` 값을 가진 사전을 반환합니다.

### 사용자 지정 미분 불가능 헤드

사용자 지정 미분 불가능 헤드는 다음 요구 사항을 따라야 합니다.

* `predict` 메서드: `(self, np.array with shape [num_inputs, embedding_size]) -> np.array with shape [num_inputs]` - 이 메서드는 임베딩을 분류합니다. 출력은 `[0, num_classes)` 범위의 정수여야 합니다.
* `predict_proba` 메서드: `(self, np.array with shape [num_inputs, embedding_size]) -> np.array with shape [num_inputs, num_classes]` - 이 메서드는 임베딩을 각 클래스에 대한 확률로 분류합니다. 각 입력에 대해 크기가 `num_classes`인 배열은 합계가 1이어야 합니다. `np.argmax(output, dim=-1)`을 적용하면 `predict`에 대한 출력이 생성됩니다.
* `fit` 메서드: `(self, np.array with shape [num_inputs, embedding_size], List[Any]) -> None` - 이 메서드는 임베딩의 `numpy` 배열과 해당 레이블 목록을 가져와야 합니다. 레이블은 반드시 정수일 필요는 없습니다. 

sklearn의 많은 분류기(예: [`RandomForestClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier), [`MLPClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html#sklearn.neural_network.MLPClassifier), [`KNeighborsClassifier`](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier) 등)는 이미 이러한 요구 사항을 충족합니다.

사용자 지정 (미분 불가능) 분류 헤드를 사용하여 SetFit 모델을 초기화할 때는 일반 `__init__` 메서드를 사용하는 것이 좋습니다.

In [ ]:
from setfit import SetFitModel
from sklearn.svm import LinearSVC
from sentence_transformers import SentenceTransformer

# 새로운 SetFit 모델 초기화
model_body = SentenceTransformer("BAAI/bge-small-en-v1.5")
model_head = LinearSVC()
model = SetFitModel(model_body, model_head)

그런 다음 일반적인 방식으로 훈련과 추론을 시작할 수 있습니다. 예:

In [ ]:
from setfit import Trainer, TrainingArguments, sample_dataset
from datasets import load_dataset

# 데이터셋 준비
dataset = load_dataset("SetFit/sst2")
train_dataset = sample_dataset(dataset["train"], label_column="label", num_samples=32)
test_dataset = dataset["test"]

# 훈련 인수 준비
args = TrainingArguments(
    batch_size=32,
    num_epochs=3,
)

# 트레이너 준비
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
)
trainer.train()

# 평가
metrics = trainer.evaluate(test_dataset)
print(metrics)
# => {'accuracy': 0.8638110928061504}

# 추론 수행
preds = model.predict([
    "매력적이고 종종 감동적인 여정입니다.",
    "매우, 매우 느립니다.",
    "때로는 지루한 영화입니다.",
])
print(preds)
# => tensor([1, 0, 0], dtype=torch.int32)